In [1]:
import warnings

warnings.filterwarnings("ignore")
import os
import pandas as pd
import janitor
import sidetable

pd.set_option("display.max_columns", 100)

from utils.data_utils import pandas_to_tex

FIGSAVEDIR = "../figs"
DATAPATH = "../adult-data"

In [2]:
# (fold cell) Load and prep web browsing data
# output = df with visits to adult sites
usecols_web_desktop = [
    "caseid",
    "category",
    "private_domain",
    "visit_duration",
    "visit_time_local",
]
usecols = [
    "caseid",
    "category",
    "private_domain",
    "page_duration",
    "session_start_time",
]

df = pd.concat(
    [
        # Get web_mobile
        pd.read_csv(
            os.path.join(
                DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"
            ),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web_desktop
        pd.read_csv(
            os.path.join(
                DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"
            ),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (
            pd.read_csv(
                os.path.join(
                    DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"
                ),
                usecols=usecols,
                low_memory=False,
            )
            # Renaming columns to be consistent w/ web_mobile & web_desktop
            .rename_column("session_start_time", "visit_time_local").rename_column(
                "page_duration", "visit_duration"
            )
        ),
    ]
)
print("output = df with web browses")
print(f"{len(df)=:,}")
df.head()

output = df with web browses
len(df)=6,297,382


,caseid,private_domain,category,visit_time_local,visit_duration
0,205323077,google.com,Search Engines and Portals,2022-05-31 23:52:37,2
1,205323077,coupons.com,"Business, Shopping",2022-06-01 01:07:35,457
2,205323077,google.com,Business,2022-06-01 01:15:12,55
3,205323077,coupons.com,"Business, Shopping",2022-06-01 01:16:07,2225
4,205323077,google.com,Search Engines and Portals,2022-06-01 04:38:10,10


## Most common categories

In [3]:
df_top10cat = df.stb.freq(["category"]).head(10)
top10cat = df_top10cat["category"].tolist()
df_top10cat

,category,count,percent,cumulative_count,cumulative_percent
0,Business,720930,12.958792,720930,12.958792
1,Search Engines and Portals,670710,12.056082,1391640,25.014874
2,Chat and Instant Messaging,613394,11.025821,2005034,36.040696
3,"Business, Social Networking",443281,7.968022,2448315,44.008718
4,Shopping,328300,5.901227,2776615,49.909945
5,"Business, Information Technology",284428,5.112623,3061043,55.022568
6,"Entertainment, Streaming Media",261165,4.694468,3322208,59.717036
7,"News and Media, Search Engines and Portals",231157,4.155071,3553365,63.872107
8,"Business, Education",181106,3.255399,3734471,67.127506
9,"Business, Shopping",135431,2.434386,3869902,69.561893


## Get examples of top 10

In [4]:
sites_column = []
for cat in top10cat:
    #     print(cat)
    _df = (
        df.query(f"category=='{cat}'")
        .groupby("private_domain")
        .size()
        .reset_index()
        .sort_values(0, ascending=False)
        .head(3)
    )
    sites = ", ".join(_df["private_domain"].tolist())
    sites = "(" + sites + ")"
    sites_column.append(sites)
#     display(_df)

In [5]:
df_top10cat["examples"] = sites_column
df_top10cat

,category,count,percent,cumulative_count,cumulative_percent,examples
0,Business,720930,12.958792,720930,12.958792,"(decipherinc.com, samplicio.us, privatelink.de)"
1,Search Engines and Portals,670710,12.056082,1391640,25.014874,"(google.com, google.co.uk, yahoo.com)"
2,Chat and Instant Messaging,613394,11.025821,2005034,36.040696,"(google.com, yahoo.com, live.com)"
3,"Business, Social Networking",443281,7.968022,2448315,44.008718,"(facebook.com, facebook.co, soocial.com)"
4,Shopping,328300,5.901227,2776615,49.909945,"(amazon.com, ebay.com, walmart.com)"
5,"Business, Information Technology",284428,5.112623,3061043,55.022568,"(clarity.ms, sentry.io, inboxdollars.com)"
6,"Entertainment, Streaming Media",261165,4.694468,3322208,59.717036,"(youtube.com, hulu.com, netflix.com)"
7,"News and Media, Search Engines and Portals",231157,4.155071,3553365,63.872107,"(bing.com, att.net)"
8,"Business, Education",181106,3.255399,3734471,67.127506,"(yougov.com, google.com, prolific.co)"
9,"Business, Shopping",135431,2.434386,3869902,69.561893,"(amazon.com, rakuten.com, instacart.com)"


## Duration on sites

In [6]:
df_ind = df.groupby(["caseid", "category"])["visit_duration"].sum().reset_index()
df_ind.head(3)

,caseid,category,visit_duration
0,47541,"Advertising, Business",221
1,47541,"Advertising, Business, Information Technology",126
2,47541,"Blogs and Personal Sites, Social Networking",2


In [7]:
df_describe = pd.DataFrame()
ntiles = [0.25, 0.5, 0.75, 0.9, 0.95]
for cat in top10cat:
    #     print(cat)
    # Get cat
    _df = (
        df_ind.query(f"category=='{cat}'")
        .assign(visit_hours=lambda df_: df_["visit_duration"] / 3600)
        .select_columns("visit_hours")
    )
    # Describe cat
    _df = (
        _df.describe(percentiles=ntiles)
        .T.reset_index(drop=True)
        .assign(category=cat)
        .remove_columns(["count"])
    )
    df_describe = pd.concat([df_describe, _df])
#     _df = _df["visit_hours"].describe()

df_describe

,mean,std,min,25%,50%,75%,90%,95%,max,category
0,4.828540,10.027118,0.0,0.529028,1.801944,5.006389,11.715389,18.705833,140.503333,Business
0,4.134727,8.883785,0.0,0.231111,1.452222,4.851944,10.239222,15.412333,139.885556,Search Engines and Portals
0,8.998323,25.792134,0.0,0.222361,2.653750,9.697500,21.336833,31.136986,492.304444,Chat and Instant Messaging
0,6.434685,18.590709,0.0,0.063750,0.617222,4.537986,15.418056,33.022292,245.533333,"Business, Social Networking"
0,3.635872,7.297028,0.0,0.227917,1.182222,3.762083,9.041111,15.865806,78.976389,Shopping
0,1.834694,4.855313,0.0,0.120972,0.529167,1.776528,4.115722,6.968556,87.757778,"Business, Information Technology"
0,7.151632,32.694566,0.0,0.061944,0.340000,2.972917,14.780278,27.789653,617.479167,"Entertainment, Streaming Media"
0,3.191887,6.277304,0.0,0.035000,0.700000,3.792222,8.988889,16.033889,61.792778,"News and Media, Search Engines and Portals"
0,1.827394,4.871890,0.0,0.201667,0.685000,1.811667,3.752611,6.520556,104.547778,"Business, Education"
0,1.454859,3.177598,0.0,0.114792,0.438333,1.298194,3.490000,5.921403,32.187222,"Business, Shopping"


## Combine

In [8]:
df_top10cat = (
    df_top10cat.merge(df_describe, how="left", on="category", validate="1:1")
    .round(1)
    .assign(
        **{
            k: lambda df_, col=k: df_[col].apply(lambda x: "{:,}".format(x))
            for k in ["count", "cumulative_count"]
        }
    )
)
pandas_to_tex(df_top10cat, "../tabs/top10cat_description")
df_top10cat

,category,count,percent,cumulative_count,cumulative_percent,examples,mean,std,min,25%,50%,75%,90%,95%,max
0,Business,"720,930",13.0,"720,930",13.0,"(decipherinc.com, samplicio.us, privatelink.de)",4.8,10.0,0.0,0.5,1.8,5.0,11.7,18.7,140.5
1,Search Engines and Portals,"670,710",12.1,"1,391,640",25.0,"(google.com, google.co.uk, yahoo.com)",4.1,8.9,0.0,0.2,1.5,4.9,10.2,15.4,139.9
2,Chat and Instant Messaging,"613,394",11.0,"2,005,034",36.0,"(google.com, yahoo.com, live.com)",9.0,25.8,0.0,0.2,2.7,9.7,21.3,31.1,492.3
3,"Business, Social Networking","443,281",8.0,"2,448,315",44.0,"(facebook.com, facebook.co, soocial.com)",6.4,18.6,0.0,0.1,0.6,4.5,15.4,33.0,245.5
4,Shopping,"328,300",5.9,"2,776,615",49.9,"(amazon.com, ebay.com, walmart.com)",3.6,7.3,0.0,0.2,1.2,3.8,9.0,15.9,79.0
5,"Business, Information Technology","284,428",5.1,"3,061,043",55.0,"(clarity.ms, sentry.io, inboxdollars.com)",1.8,4.9,0.0,0.1,0.5,1.8,4.1,7.0,87.8
6,"Entertainment, Streaming Media","261,165",4.7,"3,322,208",59.7,"(youtube.com, hulu.com, netflix.com)",7.2,32.7,0.0,0.1,0.3,3.0,14.8,27.8,617.5
7,"News and Media, Search Engines and Portals","231,157",4.2,"3,553,365",63.9,"(bing.com, att.net)",3.2,6.3,0.0,0.0,0.7,3.8,9.0,16.0,61.8
8,"Business, Education","181,106",3.3,"3,734,471",67.1,"(yougov.com, google.com, prolific.co)",1.8,4.9,0.0,0.2,0.7,1.8,3.8,6.5,104.5
9,"Business, Shopping","135,431",2.4,"3,869,902",69.6,"(amazon.com, rakuten.com, instacart.com)",1.5,3.2,0.0,0.1,0.4,1.3,3.5,5.9,32.2
